<a href="https://colab.research.google.com/github/kterra/tidd-ia-aplicada/blob/main/src/Teaching_Assistant_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Criado por [Kizzy Terra](https://www.linkedin.com/in/kizzyterra/)


# Implementação de Crew para Assistência Pedagógica

In [58]:
!pip install -q -U crewai crewai_tools

In [59]:
from crewai import Agent, Task, Crew

In [60]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [61]:
lessonPlanner = Agent(
    role='Preparador de aulas',
    name='Lesson Planner',
    goal='Elabore um plano de aula e um roteiro de execução para uma aula com {tema}. Este plano de aula deve sugerir três opções de atividades de metodologias ativas para serem feitas durante o tempo {tempo_ativo}.',
    backstory = 'Você está trabalhando como assistente de uma professora de graduação em uma faculdade de computação.'
                'Você vai elaborar planos de aula a partir do {tema} da aula e do tempo de duração da aula e do tempo de duração da aula {tempo}.'
                'Seu trabalho servirá como apoio para a professora Kizzy Terra.',
    llm_model='gpt-4o',
    verbose=True
)

generate_plan = Task(
    description='Elabore um plano de aula e um roteiro de execução para uma aula com {tema}.',
    agent=lessonPlanner,
    expected_output='Plano de aula com tema da aula e roteiro de execução.'
)

In [62]:
slidesGenerator = Agent(
    role='Preparador de aulas',
    name='Slides Generator',
    goal='Elabore os slides correspondentes ao plano de aula criado.',
    backstory = 'Você está trabalhando como assistente de uma professora de graduação em uma faculdade de computação.'
                'Você vai os slides de aula a partir do {tema} da aula..',
    llm_model='gpt-4o',
    verbose=True
)

generate_slides = Task(
    description='Elabore os slides correspondentes ao plano de aula criado para aula com {tema}.',
    agent=lessonPlanner,
    expected_output='Slides com numeração, conteúdo, referências, sugestões de imagens.'
)

In [63]:
kahootGenerator = Agent(
    role='Gerador de kahoot para aula.',
    name='Kahoot Generator',
    goal='Elabore um quiz kahoot que avalie a compreensão dos conceitos abordados em aula.',
    backstory = 'Você está trabalhando como assistente de uma professora de graduação em uma faculdade de computação.'
                'Você vai elaborar um kahoot para o tema {tema} da aula.',
    llm_model='gpt-4o',
    verbose=True
)

generate_kahoot = Task(
    description='Elabore um quiz kahoot que avalie a compreensão dos conceitos abordados em aula.',
    agent=kahootGenerator,
    expected_output='Questões do kahoot.'
)

In [64]:
testGenerator = Agent(
    role='Gerador de questões de prova',
    name='Test Generator',
    goal='Elabore questões de prova no estilo ENADE. As questões devem ser do tipo: multipla escolha, multipla escolha complexa com quatro afirmações para que o aluno identifique quais estão corretas, asserção-razão, relacionar colunas. As questões devem ter um texto-base com contexto e motivação, 5 alternativas e apenas uma correta. Indique o nivel de dificuldade e a taxonomia de bloom. Elabore também a solução da questão justificando porque cada alternativa está correta ou incorreta. Evite questões fáceis.',
    backstory = 'Você está trabalhando como assistente de uma professora de graduação em uma faculdade de computação.'
                'Você vai elaborar questões de prova para o tema {tema} da aula.'
                'Seu trabalho servirá como apoio para a professora Kizzy Terra.',
    llm_model='gpt-4o',
    verbose=True
)

generate_test = Task(
    description='Elabore questões de prova no estilo ENADE. As questões devem ser do tipo: multipla escolha, asserção-razão, relacionar colunas. As questões devem ter solução.',
    agent=testGenerator,
    expected_output='Lista de questões de prova.'
)

## Crew

In [65]:
equipe = Crew(
    agents=[lessonPlanner, slidesGenerator, kahootGenerator, testGenerator],
    tasks=[
        generate_plan,# Geração do plano de aula
        generate_slides, # Geração dos slides
        generate_kahoot,
        generate_test    # Geração das questões
    ],
    verbose=True
)

In [66]:
tema = 'IOT - Processamento de sinal analógico com Arduino e ESP32'
duracao = '1h e 45 minutos'
ativo = '25 minutos'
entradas = { "tema": tema,
            "tempo": duracao,
             "tempo_ativo": ativo}

# tema = 'Introdução ao Aprendizado de Máquina'
# duracao = '180 minutos'
# ativo = '60 minutos'
# entradas = { "tema": tema,
#             "tempo": duracao,
#              "tempo_ativo": ativo}

In [67]:
result = equipe.kickoff(inputs=entradas)

# Agent: Preparador de aulas
## Task: Elabore um plano de aula e um roteiro de execução para uma aula com IOT - Processamento de sinal analógico com Arduino e ESP32.


# Agent: Preparador de aulas
## Final Answer: 
**Plano de Aula: IOT - Processamento de Sinal Analógico com Arduino e ESP32**

**Tema da Aula:** Introdução ao Processamento de Sinal Analógico com Arduino e ESP32

**Objetivos da Aula:**
1. Compreender o conceito de sinais analógicos e sua importância em projetos de IoT.
2. Aprender a utilizar o Arduino e o ESP32 para processar sinais analógicos.
3. Implantar um projeto prático utilizando sensores analógicos.

**Duração da Aula:** 1h e 45 minutos

**Roteiro de Execução:**

**1. Introdução (15 minutos)**
   - Apresentação do tema da aula e seus objetivos.
   - Breve explicação sobre o que são sinais analógicos e a sua relevância em aplicações de IoT.
   - Apresentação dos dispositivos Arduino e ESP32, destacando suas características e diferenças.

**2. Teoria (30 minutos)**


In [68]:
lesson_planned = generate_plan.output
slides_generated = generate_slides.output
kahoot_generated = generate_kahoot.output
test_generated = generate_test.output

In [69]:
# Formatar resultados em Markdown
markdown_result = f"""
# Resultados

*Este material foi gerado por uma CrewAI*

## Sugestão de Plano de Aula de {entradas['tema']}
**Duração da Aula:** {entradas['tempo']}
**Tempo de Atividades Ativas:** {entradas['tempo_ativo']}

---

{lesson_planned}

## Sugestão de Slides para Aula

{slides_generated}

## Sugestão de Kahoot

{kahoot_generated}

## Sugestão de Questões de Prova (Padrão ENADE)

{test_generated}

"""



In [70]:
file_path_result = "/content/resultado-{tema}.md".format(tema=entradas['tema'])


# Salvar o Markdown
with open(file_path_result, "w") as file:
    file.write(markdown_result)


print(f"Arquivos Markdown salvo em: {file_path_result}")

from google.colab import files

# Baixar o arquivo
files.download(file_path_result)


Arquivos Markdown salvo em: /content/resultado-IOT - Processamento de sinal analógico com Arduino e ESP32.md


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [71]:
# reviewerAgent = Agent(
#     role='Revisor de materiais',
#     name='Reviewer',
#     goal='Revisar os materiais gerados, incluindo planos de aula e questões de prova. O foco deve ser identificar e corrigir problemas de gramática, coerência, clareza e alinhamento pedagógico. Ofereça sugestões detalhadas para melhorias.',
#     backstory='Você trabalha como assistente pedagógico para a professora Kizzy Terra e é responsável por garantir a qualidade dos materiais educacionais.',
#     llm_model='gpt-4',
#     verbose=True
# )

# # Nova Tarefa de Revisão do Plano de Aula
# review_plan = Task(
#     description='Revise o plano de aula gerado, identificando problemas e sugerindo melhorias. Gere um plano de aula atualizado com as melhorias identificadas. Acrescente, ao final, as modificações feitas.',
#     agent=reviewerAgent,
#     expected_output='Plano de aula revisado, incluindo, ao final, uma seção com as modificações feitas com justificativa.'
# )

# # Nova Tarefa de Revisão das Questões de Prova
# review_test = Task(
#     description='Revise as questões de prova geradas, verificando gramática, clareza e alinhamento pedagógico. Verifique se as questões estão no padrão ENADE. Verifique se a solução e o nível da questão estão corretos. Gere a lista de questões de prova atualizada com as melhorias identificadas. Acrescente, ao final, as modificações feitas.',
#     agent=reviewerAgent,
#     expected_output='Lista de questões de prova revisada, incluindo uma seção com as modificações feitas com justificativa.'
# )


In [72]:

# markdown_reviewed_result = f"""
# # Resultados Revisados

# *Este material foi gerado e revisado por uma CrewAI*

# ## Tema: {entradas['tema']}
# **Duração da Aula:** {entradas['tempo']}
# **Tempo de Atividades Ativas:** {entradas['tempo_ativo']}

# ---

# {reviewed_lesson_planned}

# ---

# {reviewed_test_generated}

# """

# file_path_reviewed_result = "/content/resultado-revisado-{tema}.md".format(tema=entradas['tema'])

# with open(file_path_reviewed_result, "w") as file:
#     file.write(markdown_reviewed_result)
